In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:

# Load the stock price data
stock_data = pd.read_csv("GOOG_stock.csv")
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_data.set_index('Date', inplace=True)

# Use only the 'Close' price for prediction
data = stock_data[['Close']].values

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)


In [ ]:

# Prepare data for LSTM
sequence_length = 14  # Number of time steps to look back
X = []
y = []
for i in range(len(data_scaled) - sequence_length):
    X.append(data_scaled[i:i+sequence_length])
    y.append(data_scaled[i+sequence_length])
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


In [ ]:

# Build the LSTM model
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1))
model.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, batch_size=128, epochs=80, validation_data=(X_test, y_test))

# Make predictions
predicted = model.predict(X_test)
predicted = scaler.inverse_transform(predicted)


In [ ]:

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(stock_data.index[train_size+sequence_length:], stock_data['Close'][train_size+sequence_length:], label='Actual')
plt.plot(stock_data.index[train_size+sequence_length:], predicted, label='Predicted')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.title('GOOG Stock Price Prediction')
plt.show()